# Gaussian Mixture Models 高斯混合模型

本节将讨论使用EM(Expectation-Maximization)算法用来做密度预估。

我们希望通过联合分布 $p(x^{(i)},z^{(i)})=p(x^{(i)}|z^{(i)})p(z^{(i)})$ 对数据进行建模。其中，$z^{(i)} \sim Multinomial(\phi) （\phi_j \geq 0, \sum_{j=1}^k\phi_j=1,\phi_j=p(z^{(i)}=j)）$，并且 $x^{(i)}|z^{(i)}=j \sim \mathcal{N}(\mu_j,\Sigma_j)$。这里，$k$ 是 $z^{(i)}$ 所有可能取值的数量。所以，每个数据 $x^{(i)}$ 产生的过程是首先从 $\{1,\cdots,k\}$ 中随机挑选出 $z^{(i)}$，之后根据 $z^{(i)}$ 所对应的 $k$ 元高斯分布生成 $x^{(i)}$。这个模型叫做高斯混合模型。注意到，$z^{(i)}$ 在这里是**隐式 latent**随机变量，它或者是隐藏或者是无法直接观测的。

根据上述定义，模型的参数为 $\phi,\mu,\Sigma$，为了进行参数估计，根据数据可以写出以下似然函数：
$$
\begin{split}
\ell(\phi,\mu,\Sigma) &= \sum_{i=1}^m log p(x^{(i)};\phi,\mu,\Sigma) \\
&= \sum_{i=1}^m log \sum_{z^{(i)}=1}^k p(x^{(i)}|z^{(i)};\mu,\Sigma)p(z^{(i)};\phi)
\end{split}
$$
然而，如果我们对上面这个公式，令各偏导为零，是无法解出的。

这里随机变量 $z^{(i)}$ 指定了对应的数据 $x^{(i)}$ 来自 $k$ 个高斯分布中的具体哪一个。如果事先知道 $z^{(i)}$ 是哪一个，最大似然估计法就容易解出，因为其形式可以写作
$$
\ell(\phi,\mu,\Sigma) = \sum_{i=1}^m logp(x^{(i)}|z^{(i)};\mu,\Sigma)+logp(z^{(i)};\phi)
$$
上面这个似然函数取最大值时各个参数分别为
$$
\begin{split}
\phi_j &= \frac{1}{m}\sum_{i=1}^m 1\{z^{(i)=j}\}, \\
\mu_j &= \frac{\sum_{i=1}^m 1\{z^{(i)}=j\}x^{(i)}}{\sum_{i=1}^m 1\{z^{(i)}=j\}}, \\
\Sigma_j &= \frac{\sum_{i=1}^m 1\{z^{(i)}=j\}(x^{(i)}-\mu_j)(x^{(i)}-\mu_j)^T}{\sum_{i=1}^m 1\{z^{(i)}=j\}}
\end{split}
$$

没错，可以看到，如果 $z^{(i)}$ 事先知道，那么这里的最大似然估计法就和高斯判别分析模型的十分类似，$z^{(i)}$ 在这里起到的就是数据标签的作用。（微小的区别是，这里 $z^{(i)}$ 服从多项分布而不是伯努利分布，以及每个高斯分布在这里可以有不同的协方差）

但是在密度预测问题中，$z^{(i)}$ 是未知的。EM算法包括两个主要迭代步骤：1）E步骤，猜测 $z^{(i)}$ 的值；2）M步骤，根据步骤1)中的猜测，更新模型参数。在M步骤中，由于我们假设E步骤中猜测的值是正确的，求解最大化非常容易。具体的算法步骤如下，重复直至收敛：
1. E步骤，对每组 $i,j$，令 $w_j^{(i)}=p(z^{(i)}=j|x^{(i)};\phi,\mu,\Sigma)$
2. M步骤，更新参数
$$
\begin{split}
\phi_j &= \frac{1}{m}\sum_{i=1}^m w_j^{(i)}, \\
\mu_j &= \frac{\sum_{i=1}^m w_j^{(i)}x^{(i)}}{\sum_{i=1}^m w_j^{(i)}}, \\
\Sigma_j &= \frac{\sum_{i=1}^m w_j^{(i)}(x^{(i)}-\mu_j)(x^{(i)}-\mu_j)^T}{\sum_{i=1}^m w_j^{(i)}}
\end{split}
$$

在E步骤中，我们计算了在当前参数 $\phi,\mu,\Sigma$ 下，给定数据 $x^{(i)}$ 时，$z^{(i)}$ 的后验概率，即
$$ p(z^{(i)}=j|x^{(i)};\phi,\mu,\Sigma) = \frac{p(x^{(i)}|z^{(i)}=j;\mu,\Sigma)p(z^{(i)}=j;\phi)}{\sum_{l=1}^k p(x^{(i)}|z^{(i)}=l;\mu,\Sigma)p(z^{(i)}=l;\phi)} $$

EM算法容易收到局部极值的影响，所以根据多个不同的初始猜测值去计算是常见的手段。EM算法的可解释性非常强，我们通过重复的猜测和重新计算，来得到最终结果。关于收敛性的保证，将在EM的专题中详细介绍。